In [1]:
import dask.dataframe as dd
import numpy as np
import dask
from dask_ml.preprocessing import OneHotEncoder
from dask_ml.impute import SimpleImputer
from dask_ml.preprocessing import StandardScaler
from dask_ml.compose import ColumnTransformer
#Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
#importing datasets
headers_orig = open('headers_orig.txt', 'r')
headers_orig = headers_orig.read().strip().split('\n')
orig = dd.read_csv("sample_orig_2018.txt", names=headers_orig, sep="|",sample=25000000)

headers_svcg = open('headers_svcg.txt', 'r')
headers_svcg = headers_svcg.read().strip().split('\n')
svcg = dd.read_csv("sample_svcg_2018.txt", names=headers_svcg, sep="|",sample=25000000)
svcg = svcg[['Loan Sequence Number', 'Zero Balance Code', 'Current Loan Delinquency Status']]

In [4]:
#join datasets, remove duplicates
orig = svcg.join(orig.set_index("Loan Sequence Number"), on="Loan Sequence Number", how="left")
orig = orig.drop_duplicates(subset='Loan Sequence Number', keep='last')

# Preprocessing

In [5]:
#drop unnecessary features
drop_features = ['First Payment Date', 
                 'Maturity Date', 
                 'Metropolitan Statistical Area (MSA) Or Metropolitan Division','Pre-HARP Loan Sequence Number',
                 'Property State',
                 'Postal Code',
                 'Loan Sequence Number',
                 'Seller Name',
                 'Servicer Name',
                 'Pre-HARP Loan Sequence Number',
                 'Property Valuation Method']
orig = orig.drop(drop_features, axis=1)
orig = orig[orig['HARP Indicator'] != 'Y']
orig = orig.drop('HARP Indicator', axis=1)

In [6]:
#impute features with N/A values (such as 99, 9, etc.)
simpleImputer = SimpleImputer(missing_values=99, strategy='constant', fill_value=1)
orig['Number of Units'] = simpleImputer.fit_transform(orig[['Number of Units']])['Number of Units']

simpleImputer = SimpleImputer(missing_values='9', strategy='constant', fill_value='P')
orig['Occupancy Status'] = simpleImputer.fit_transform(orig[['Occupancy Status']])['Occupancy Status']

simpleImputer = SimpleImputer(missing_values=999, strategy='constant', fill_value=65)
orig['Original Debt-to-Income (DTI) Ratio'] = simpleImputer.fit_transform(orig[['Original Debt-to-Income (DTI) Ratio']])['Original Debt-to-Income (DTI) Ratio']

simpleImputer = SimpleImputer(missing_values=999, strategy='constant', fill_value=np.NaN)
orig['Original Loan-to-Value (LTV)'] = simpleImputer.fit_transform(orig[['Original Loan-to-Value (LTV)']])['Original Loan-to-Value (LTV)']
simpleImputer = SimpleImputer(strategy='mean', fill_value=1)
orig['Original Loan-to-Value (LTV)'] = simpleImputer.fit_transform(orig[['Original Loan-to-Value (LTV)']])['Original Loan-to-Value (LTV)']

simpleImputer = SimpleImputer(missing_values='T', strategy='constant', fill_value='R')
orig['Channel'] = simpleImputer.fit_transform(orig[['Channel']])['Channel']
simpleImputer = SimpleImputer(missing_values='9', strategy='constant', fill_value='R')
orig['Channel'] = simpleImputer.fit_transform(orig[['Channel']])['Channel']

simpleImputer = SimpleImputer(missing_values=9, strategy='constant', fill_value=1)
orig['Prepayment Penalty Mortgage (PPM) Flag'] = simpleImputer.fit_transform(orig[['Prepayment Penalty Mortgage (PPM) Flag']])['Prepayment Penalty Mortgage (PPM) Flag']

simpleImputer = SimpleImputer(missing_values='99', strategy='constant', fill_value='SF')
orig['Property Type'] = simpleImputer.fit_transform(orig[['Property Type']])['Property Type']

simpleImputer = SimpleImputer(missing_values='9', strategy='constant', fill_value='P')
orig['Loan Purpose'] = simpleImputer.fit_transform(orig[['Loan Purpose']])['Loan Purpose']

simpleImputer = SimpleImputer(missing_values=9, strategy='constant', fill_value=1)
orig['Number of Borrowers'] = simpleImputer.fit_transform(orig[['Number of Borrowers']])['Number of Borrowers']

In [7]:
#binarize features with 2 values
orig['Prepayment Penalty Mortgage (PPM) Flag'] = orig['Prepayment Penalty Mortgage (PPM) Flag'].replace({'Y': 1, 'N': 0})

orig['Amortization Type (Formerly Product Type)'] = orig['Amortization Type (Formerly Product Type)'].replace({'FRM': 1, 'ARM': 0})

simpleImputer = SimpleImputer(strategy='constant', fill_value='N')
orig['Super Conforming Flag'] = simpleImputer.fit_transform(orig[['Super Conforming Flag']])['Super Conforming Flag']
orig['Super Conforming Flag'] = orig['Super Conforming Flag'].replace({'Y': 1, 'N': 0})

orig['Program Indicator'] = orig['Program Indicator'].replace({'H': 1, '9': 0})

orig['Interest Only (I/O) Indicator'] = orig['Interest Only (I/O) Indicator'].replace({'Y': 1, 'N': 0})

In [8]:
#onehot encode features
onehot_features = ['First Time Homebuyer Flag', 
                   'Occupancy Status', 
                   'Channel',
                   'Property Type',
                   'Loan Purpose']

orig['First Time Homebuyer Flag'] = orig['First Time Homebuyer Flag'].astype("category")
orig['Occupancy Status'] = orig['First Time Homebuyer Flag'].astype("category")
orig['Channel'] = orig['First Time Homebuyer Flag'].astype("category")
orig['Property Type'] = orig['First Time Homebuyer Flag'].astype("category")
orig['Loan Purpose'] = orig['First Time Homebuyer Flag'].astype("category")
orig = orig.categorize(columns=['First Time Homebuyer Flag','Occupancy Status','Channel','Property Type','Loan Purpose'])

onehot_transformer = OneHotEncoder(sparse=False)

#scale numerical features
numerical_features = ['Credit Score', 
                      'Mortgage Insurance Percentage (MI %)', 
                      'Number of Units', 
                      'Original Combined Loan-to-Value (CLTV)', 
                      'Original Debt-to-Income (DTI) Ratio', 
                      'Original UPB', 
                      'Original Loan-to-Value (LTV)',
                      'Original Interest Rate',
                      'Original Loan Term',
                      'Number of Borrowers']
numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer(transformers=[
    ('onehot', onehot_transformer, onehot_features), 
    ('numerical', numerical_transformer, numerical_features)],
    remainder='passthrough'
)

orig = preprocessor.fit_transform(orig)

First Time Homebuyer Flag_9  First Time Homebuyer Flag_N  \
21                           1.0                          0.0   
48                           0.0                          1.0   
74                           1.0                          0.0   
109                          0.0                          0.0   
144                          1.0                          0.0   
159                          1.0                          0.0   
194                          1.0                          0.0   
223                          1.0                          0.0   
254                          1.0                          0.0   
289                          0.0                          0.0   

     First Time Homebuyer Flag_Y  Occupancy Status_9  Occupancy Status_N  \
21                           0.0                 1.0                 0.0   
48                           0.0                 0.0                 1.0   
74                           0.0                 1.0                 0.0   
109                          1.0                 0.0                 0.0   
144                          0.0                 1.0                 0.0   
159                          0.0                 1.0                 0.0   
194                          0.0                 1.0                 0.0   
223                          0.0                 1.0                 0.0   
254                          0.0                 1.0                 0.0   
289                          1.0                 0.0                 0.0   

     Occupancy Status_Y  Channel_9  Channel_N  Channel_Y  Property Type_9  \
21                  0.0        1.0        0.0        0.0              1.0   
48                  0.0        0.0        1.0        0.0              0.0   
74                  0.0        1.0        0.0        0.0              1.0   
109                 1.0        0.0        0.0        1.0              0.0   
144                 0.0        1.0        0.0        0.0              1.0   
159                 0.0        1.0        0.0        0.0              1.0   
194                 0.0        1.0        0.0        0.0              1.0   
223                 0.0        1.0        0.0        0.0              1.0   
254                 0.0        1.0        0.0        0.0              1.0   
289                 1.0        0.0        0.0        1.0              0.0   

     ...  Original Interest Rate  Original Loan Term  Number of Borrowers  \
21   ...               -0.236655            0.416217            -0.925756   
48   ...               -3.117368           -2.527024             1.080198   
74   ...               -2.069836            0.416217            -0.925756   
109  ...               -1.022304            0.416217            -0.925756   
144  ...               -1.807953            0.416217             1.080198   
159  ...               -1.284187            0.416217            -0.925756   
194  ...               -1.284187            0.416217            -0.925756   
223  ...               -0.498538            0.416217             1.080198   
254  ...               -0.236655            0.416217             1.080198   
289  ...               -1.807953            0.416217             1.080198   

     Zero Balance Code  Current Loan Delinquency Status  \
21                 1.0                                0   
48                 1.0                                0   
74                 1.0                                0   
109                NaN                                0   
144                NaN                                0   
159                1.0                                0   
194                NaN                                0   
223                1.0                                0   
254                1.0                                0   
289                NaN                                2   

     Prepayment Penalty Mortgage (PPM) Flag  \
21                                        0   
48          

In [9]:
#fix label generating columns
orig['Current Loan Delinquency Status'] = orig['Current Loan Delinquency Status'].replace({'R': 9})
orig['Current Loan Delinquency Status'] = orig['Current Loan Delinquency Status'].astype('int32')

In [12]:
#output as df, fix column names
output = orig.compute()
output.columns = output.columns.str.replace('_', '')
output.columns = output.columns.str.replace(' ', '')
output.columns = output.columns.str.replace('(', '')
output.columns = output.columns.str.replace(')', '')

In [16]:
#export to parquet
output.to_parquet('features.parquet')